In [13]:
import numpy as np
import pandas as pd
#import user data
df_user = pd.read_csv('ratings.csv')
print(df_user)

        userId  movieId  rating   timestamp
0            1       31     2.5  1260759144
1            1     1029     3.0  1260759179
2            1     1061     3.0  1260759182
3            1     1129     2.0  1260759185
4            1     1172     4.0  1260759205
...        ...      ...     ...         ...
99999      671     6268     2.5  1065579370
100000     671     6269     4.0  1065149201
100001     671     6365     4.0  1070940363
100002     671     6385     2.5  1070979663
100003     671     6565     3.5  1074784724

[100004 rows x 4 columns]


In [14]:
#checking for null_values
null_values = df_user.isnull().sum()
#count null values for eachn column
print(null_values)

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64


In [15]:
# Creating dictionary containing each user and a list of the movies they have rated and their corresponding rating
user_movie_dict = df_user.groupby('userId').apply(lambda x: list(zip(x['movieId'], x['rating']))).to_dict()


#reduce amount of users to prevent overload in system

# items_list = list(user_movie_dict.items())


# Slice the list
# sliced_items = items_list

# Convert back to dictionary
# sliced_user_dict = dict(sliced_items)



In [16]:
#import umovie data
df_movie = pd.read_csv('movies.csv')
print(df_movie)

      movieId                                              title  \
0           1                                   Toy Story (1995)   
1           2                                     Jumanji (1995)   
2           3                            Grumpier Old Men (1995)   
3           4                           Waiting to Exhale (1995)   
4           5                 Father of the Bride Part II (1995)   
...       ...                                                ...   
9120   162672                                Mohenjo Daro (2016)   
9121   163056                               Shin Godzilla (2016)   
9122   163949  The Beatles: Eight Days a Week - The Touring Y...   
9123   164977                           The Gay Desperado (1936)   
9124   164979                              Women of '69, Unboxed   

                                           genres  
0     Adventure|Animation|Children|Comedy|Fantasy  
1                      Adventure|Children|Fantasy  
2                          

In [17]:
#checking for null_values
null_values = df_movie.isnull().sum()
#count null values for eachn column
print(null_values)

movieId    0
title      0
genres     0
dtype: int64


In [18]:
#listing number of movies
movie_id_list = df_movie['movieId'].tolist()
# movie_id_list = movie_id_list



In [19]:
nm = len(movie_id_list)
nu = len(user_movie_dict)
#stores ratings
Y = np.zeros((nm, nu))
#binary valued indicator to check for if a user voted or not
R = np.zeros((nm,nu))
#stores ratings of users for each movie
Y_i = np.zeros(nu)
#binary valued indicator to check for if a user voted or not for each movie
R_i = np.zeros(nu)

for i in range(nm):
    found = False
    #iterate through movie list
    for j, (user,movies) in enumerate(user_movie_dict.items()):
        #check if movie id is in the list of movie ids this user has rated
        for movie in movies:
            if movie[0] == movie_id_list[i]:
                #if the movie is in the list then store the ratings
                Y_i[j] = movie[1]
                R_i[j] = 1
                found = True
        if not found: #found is false
            #if the movie is not in their movie list then it is not rated by that user
            Y_i[j] = 0
            R_i[j] = 0
    Y[i] = Y_i
    R[i] = R_i
print(Y)
print(R)

        


    

        
        
    

[[0. 0. 0. ... 0. 4. 5.]
 [0. 0. 0. ... 0. 4. 5.]
 [0. 0. 0. ... 0. 4. 5.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 0. 1. 1.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [21]:
#Normazlizing Y to assume an average rating for each user rather than 0
def y_norm(y,r):
    for i in range(y.shape[0]):
        #if there is atleast one rating i.e all columns arent 0
        if y[i, r[i, :].astype(bool)].size != 0:
            mean_value = np.mean(y[i, r[i, :].astype(bool)])
            y[i, ~r[i, :].astype(bool)] = mean_value
           

    return y
Y_norm = y_norm(Y,R)
print(Y_norm)


[[3.87246964 3.87246964 3.87246964 ... 3.87246964 4.         5.        ]
 [3.69107143 3.69107143 3.69107143 ... 3.69107143 4.         5.        ]
 [3.56101695 3.56101695 3.56101695 ... 3.56101695 4.         5.        ]
 ...
 [5.         5.         5.         ... 5.         5.         5.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [22]:
#defining the input
X = df_movie["genres"].values
print(X)

['Adventure|Animation|Children|Comedy|Fantasy'
 'Adventure|Children|Fantasy' 'Comedy|Romance' ... 'Documentary' 'Comedy'
 'Documentary']


In [23]:
from sklearn.feature_extraction.text import CountVectorizer
#count vectorization
vectorizer = CountVectorizer()
x_bow = vectorizer.fit_transform(X)
x_enc = x_bow.toarray()
x_enc = x_enc.astype(float)
print(x_enc)

[[0. 1. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [24]:
m,n = x_enc.shape


In [25]:
def cofi_cost_func_v(X, W, b, Y, R, lambda_):
    """
    Returns the cost for the content-based filtering
    Vectorized for speed. Uses tensorflow operations to be compatible with custom training loop.
    Args:
      X (ndarray (num_movies,num_features)): matrix of item features
      W (ndarray (num_users,num_features)) : matrix of user parameters
      b (ndarray (1, num_users)            : vector of user parameters
      Y (ndarray (num_movies,num_users)    : matrix of user ratings of movies
      R (ndarray (num_movies,num_users)    : matrix, where R(i, j) = 1 if the i-th movies was rated by the j-th user
      lambda_ (float): regularization parameter
    Returns:
      J (float) : Cost
    """
    #make sure X and W have the same data type
    j = (tf.linalg.matmul(X, tf.transpose(W)) + b - Y)*R
    J = 0.5 * tf.reduce_sum(j**2) + (lambda_/2) * (tf.reduce_sum(X**2) + tf.reduce_sum(W**2))
    return J

In [26]:
import tensorflow as tf
from tensorflow import keras

# Set Initial Parameters (W, X), use tf.Variable to track these variables
tf.random.set_seed(1234) # for consistent results
print(nm)
print(nu)
print(n)
W = tf.Variable(tf.random.normal((nu,  n),dtype=tf.float64),  name='W')
print(W)
X = tf.Variable(tf.random.normal((nm, n),dtype=tf.float64),  name='X')
print(X.shape)
b = tf.Variable(tf.random.normal((1,nu),   dtype=tf.float64),  name='b')

# Instantiate an optimizer.
optimizer = keras.optimizers.Adam(learning_rate=1e-1)

9125
671
24
<tf.Variable 'W:0' shape=(671, 24) dtype=float64, numpy=
array([[-1.2650278 , -0.88266007,  0.45325746, ..., -0.5071518 ,
        -0.69534167,  1.78964465],
       [-0.54837295, -1.48310151,  0.38314998, ..., -0.96341192,
        -0.19556525, -0.49596018],
       [-0.15707949, -1.31670428,  0.53294877, ..., -0.25568902,
        -0.77758611,  0.93006297],
       ...,
       [ 0.41630287, -0.81200051,  1.71935755, ..., -1.84422598,
        -0.11612442, -0.52074994],
       [-0.57502626, -0.90119769,  0.4656603 , ..., -1.82202762,
        -0.53223653, -0.94507692],
       [-0.08472545,  0.0093382 , -0.7313654 , ..., -0.54756592,
         0.28952935,  0.48042513]])>
(9125, 24)


In [27]:
iterations = 200
lambda_ = 1
for iter in range(iterations):
    # Use TensorFlow’s GradientTape
    # to record the operations used to compute the cost 
    with tf.GradientTape() as tape:

        # Compute the cost (forward pass included in cost)
        cost_value = cofi_cost_func_v(x_enc, W, b, Y_norm, R, lambda_)
        print(cost_value)
        #make sure cost value data type is compatible with the requirements of tape.gradient()

    # Use the gradient tape to automatically retrieve
    # the gradients of the trainable variables with respect to the loss
    grads = tape.gradient( cost_value, [X,W,b] )

    # Run one step of gradient descent by updating
    # the value of the variables to minimize the loss.
    optimizer.apply_gradients( zip(grads, [X,W,b]) )

    # Log periodically.
    if iter % 20 == 0:
        print(f"Training loss at iteration {iter}: {cost_value:0.1f}")

tf.Tensor(5683465.426079654, shape=(), dtype=float64)
Training loss at iteration 0: 5683465.4
tf.Tensor(4941184.8460375955, shape=(), dtype=float64)
tf.Tensor(4279787.004215646, shape=(), dtype=float64)
tf.Tensor(3698139.327508841, shape=(), dtype=float64)
tf.Tensor(3193570.3537206077, shape=(), dtype=float64)
tf.Tensor(2761969.5785803683, shape=(), dtype=float64)
tf.Tensor(2397975.8088828684, shape=(), dtype=float64)
tf.Tensor(2095268.2516106183, shape=(), dtype=float64)
tf.Tensor(1846897.0630702928, shape=(), dtype=float64)
tf.Tensor(1645631.85571442, shape=(), dtype=float64)
tf.Tensor(1484286.8035570588, shape=(), dtype=float64)
tf.Tensor(1355993.4348214096, shape=(), dtype=float64)
tf.Tensor(1254408.404273454, shape=(), dtype=float64)
tf.Tensor(1173856.0264708342, shape=(), dtype=float64)
tf.Tensor(1109411.6767830148, shape=(), dtype=float64)
tf.Tensor(1056932.399887325, shape=(), dtype=float64)
tf.Tensor(1013043.4062021844, shape=(), dtype=float64)
tf.Tensor(975091.0454412941, sha

tf.Tensor(411407.6897115621, shape=(), dtype=float64)
tf.Tensor(408082.62355808204, shape=(), dtype=float64)
tf.Tensor(404904.9479278603, shape=(), dtype=float64)
tf.Tensor(401877.5427329401, shape=(), dtype=float64)
tf.Tensor(398999.7029521087, shape=(), dtype=float64)
Training loss at iteration 60: 398999.7
tf.Tensor(396266.8468283471, shape=(), dtype=float64)
tf.Tensor(393670.8378904123, shape=(), dtype=float64)
tf.Tensor(391200.77024156443, shape=(), dtype=float64)
tf.Tensor(388843.99570374936, shape=(), dtype=float64)
tf.Tensor(386587.1860454738, shape=(), dtype=float64)
tf.Tensor(384417.2994646904, shape=(), dtype=float64)
tf.Tensor(382322.3827439608, shape=(), dtype=float64)
tf.Tensor(380292.1667515262, shape=(), dtype=float64)
tf.Tensor(378318.4289617225, shape=(), dtype=float64)
tf.Tensor(376395.1110327943, shape=(), dtype=float64)
tf.Tensor(374518.190713415, shape=(), dtype=float64)
tf.Tensor(372685.3338268097, shape=(), dtype=float64)
tf.Tensor(370895.3987524721, shape=(), d

tf.Tensor(323221.0642771895, shape=(), dtype=float64)
tf.Tensor(322358.0254652929, shape=(), dtype=float64)
tf.Tensor(321507.83871721977, shape=(), dtype=float64)
tf.Tensor(320670.35808355815, shape=(), dtype=float64)
tf.Tensor(319845.4345168621, shape=(), dtype=float64)
tf.Tensor(319032.90974153095, shape=(), dtype=float64)
tf.Tensor(318232.6122773189, shape=(), dtype=float64)
tf.Tensor(317444.3565539369, shape=(), dtype=float64)
tf.Tensor(316667.94560012117, shape=(), dtype=float64)
Training loss at iteration 120: 316667.9
tf.Tensor(315903.17649239645, shape=(), dtype=float64)
tf.Tensor(315149.8470532803, shape=(), dtype=float64)
tf.Tensor(314407.7624096675, shape=(), dtype=float64)
tf.Tensor(313676.7401397976, shape=(), dtype=float64)
tf.Tensor(312956.61309481144, shape=(), dtype=float64)
tf.Tensor(312247.22970544157, shape=(), dtype=float64)
tf.Tensor(311548.45198262006, shape=(), dtype=float64)
tf.Tensor(310860.1516418601, shape=(), dtype=float64)
tf.Tensor(310182.2053369853, shap

tf.Tensor(290513.9173299295, shape=(), dtype=float64)
tf.Tensor(290153.27611168585, shape=(), dtype=float64)
tf.Tensor(289798.49545525556, shape=(), dtype=float64)
tf.Tensor(289449.48983306036, shape=(), dtype=float64)
tf.Tensor(289106.174737153, shape=(), dtype=float64)
tf.Tensor(288768.4666906271, shape=(), dtype=float64)
tf.Tensor(288436.28324051836, shape=(), dtype=float64)
tf.Tensor(288109.5429348472, shape=(), dtype=float64)
tf.Tensor(287788.1652949785, shape=(), dtype=float64)
tf.Tensor(287472.0707923895, shape=(), dtype=float64)
tf.Tensor(287161.1808363096, shape=(), dtype=float64)
tf.Tensor(286855.41777520726, shape=(), dtype=float64)
tf.Tensor(286554.70490868, shape=(), dtype=float64)
Training loss at iteration 180: 286554.7
tf.Tensor(286258.9665023577, shape=(), dtype=float64)
tf.Tensor(285968.12779695704, shape=(), dtype=float64)
tf.Tensor(285682.1150042012, shape=(), dtype=float64)
tf.Tensor(285400.8552891206, shape=(), dtype=float64)
tf.Tensor(285124.27674482315, shape=()

In [28]:
# Make a prediction using trained weights and biases
p = np.matmul(X.numpy(), np.transpose(W.numpy())) + b.numpy()
#Note: the rows represent each movie and the columns represent each user

print(p)


[[4.78301079 4.90449074 3.15575267 ... 2.06929674 4.97340879 4.20406771]
 [3.78884668 3.20290074 3.15343827 ... 5.51956631 1.6120456  3.79335014]
 [3.40355575 7.99690693 1.65664662 ... 1.59595368 5.21211661 4.17754716]
 ...
 [4.0837605  1.15774503 2.11286633 ... 2.05226433 3.6317934  3.42834999]
 [4.51080275 5.1448831  1.31747166 ... 3.52207191 3.39738848 3.68237086]
 [2.28708709 4.07145859 1.15706144 ... 2.07360663 4.03763743 3.66753127]]


In [29]:
#to pick a movie a user has not rated would be movie 1 and user 1 and based on the prediction it is most likely user 1
#will like movie 1
#Note: To find movies similar to each other, the value of the input features must be similar i.e to find a movie K similar
#to a movie i, Vector(X_i) and Vector(X_k) that are closest

print(X)

<tf.Variable 'X:0' shape=(9125, 24) dtype=float64, numpy=
array([[ 0.92169644,  0.40301904, -0.67974247, ..., -0.47733292,
         0.30809066,  0.07542342],
       [-0.45453283,  0.73742837, -0.23854552, ..., -0.74528388,
         1.38783616,  0.54653446],
       [-0.29656788,  0.40229388, -0.46436498, ..., -1.20075887,
         0.72424704,  0.10177682],
       ...,
       [ 0.88564007, -1.14287594,  1.60203391, ...,  1.02978707,
         1.24387193,  0.31709962],
       [-0.5768467 ,  0.29985087, -1.15710717, ..., -1.07555446,
        -0.61480494,  1.54751057],
       [-0.46585549,  0.92920776, -0.59434025, ...,  0.99183869,
         0.71474962,  0.59501854]])>
